In [1]:
import keras 
from keras import *
from keras.models import load_model
import numpy as np
import tensorflow as tf

import pickle
import cv2
import os.path
from utility import img_to_encoding, resize_img
from collections import defaultdict
from webcam_utility import *

Using TensorFlow backend.
C:\Users\ASUS\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [10]:
# custom loss function for model
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    # triplet formula components
    pos_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[1])) )
    neg_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[2])) )
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.maximum(basic_loss, 0.0)
   
    return loss

In [2]:
# for loading the facenet trained model 
def load_FRmodel():
    return load_model('models/model.h5', custom_objects={'triplet_loss': triplet_loss})

In [3]:
# load the saved user database
def ini_user_database():
    global user_db
    # check for existing database
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)
    else:
        # make a new one
        # we use a dict for keeping track of mapping of each person with his/her face encoding
        user_db = defaultdict(dict)
    return user_db

In [4]:
# deletes a registered user from database
def delete_user(user_db, email):
    popped = user_db.pop(email, None)

    if popped is not None:
        print('User ' + email + ' deleted successfully')
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return True
    elif popped == None:
        print('No such user !!')
        return False

In [5]:
# adds a new user face to the database using his/her image stored on disk using the image path
def add_user_img_path(user_db, FRmodel, email, name, img_path):
    if email not in user_db:
        user_db[email]['encoding'] = img_to_encoding(img_path, FRmodel)
        user_db[email]['name'] = name
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('User ' + name + ' added successfully')
        return True
    else:
        print('The name is already registered! Try a different name.........')
        return False

In [6]:
# adds a new user using image taken from webcam
def add_user_webcam(user_db, FRmodel, email, name):
    # we can use the webcam to capture the user image then get it recognized
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        if email not in user_db:
            return add_user_img_path(user_db, FRmodel, email, name, "saved_image/1.jpg")
        else:
            print('The name is already registered! Try a different name.........')
            return True
    else:
        print('There was no face found in the visible frame. Try again...........')
        return False

In [8]:
# for getting databse details
def get_details(user_db, email = None, view_all = False):
    print('Total users: ' + str(len(user_db)))
    
    user_db = ini_user_database()
    if email is  not None and email in user_db:
        print('Requested user')
        print(user_db[email])
    elif email:
        print('User not registered !')
    print()
    if view_all:
        for user in user_db.keys():
            print(user)
            print('Email: ' + str(user))
            print('Name: ' + user_db[user]['name'])
            print()

In [11]:
FRmodel = load_FRmodel()

In [12]:
user_db = ini_user_database()

In [13]:
add_user_img_path(user_db, FRmodel, 'susanta@gmail.com', 'Susanta', "saved_image/1.jpg")

TypeError: 'NoneType' object is not subscriptable

In [26]:
add_user_webcam(user_db, FRmodel, 'susanta@gmail.com', 'Susanta')

User Susanta added successfully


True

In [16]:
delete_user(user_db, 'susanta@gmail.com')

No such user !!


False

In [14]:
get_details(user_db, email = None, view_all = True)

Total users: 1

aditya@gmail.com
Email: aditya@gmail.com
Name: aditya



In [11]:
p = user_db['susanta@gmail.com'].pop('male', None)